In [1]:
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import numpy as np
import pandas as pd
import os

In [2]:
narrative_path = '../data_to_model/keras_data/model_df_full_narrative.csv'
train_path = '../data_to_model/Train and Test Sets/train_df_count.csv'

In [3]:
narrative_df = pd.read_csv(narrative_path)
narrative_df.drop(columns= ['primary_mechanism'], inplace = True )

In [4]:
train_df = pd.read_csv(train_path).merge(narrative_df,how='left', on=['cpsc_case_number'] )
#train_df

In [5]:
text_np = np.array(train_df['narrative'])
train_labels_np = np.array(train_df['primary_mechanism'])
#train_labels_np_bin = np.where(train_labels_np == 'Multi-sport', 'Multi-sport', 'Other')
print(train_labels_np[0:30])
#print(train_labels_np_bin[0:30])

['cardio/sync' 'Hit with various obj' 'cardio/sync' 'cardio/sync'
 'cardio/sync' 'cardio/sync' 'cardio/sync' 'cardio/sync'
 'Hit with various obj' 'cardio/sync' 'Undetermined/unknown' 'cardio/sync'
 'Undetermined/unknown' 'Undetermined/unknown' 'Undetermined/unknown'
 'Hit with various obj' 'cardio/sync' 'cardio/sync' 'Undetermined/unknown'
 'Multi-sport' 'Undetermined/unknown' 'cardio/sync' 'cardio/sync'
 'cardio/sync' 'Multi-sport' 'Multi-sport' 'Undetermined/unknown'
 'cardio/sync' 'cardio/sync' 'cardio/sync']


In [6]:
#l_list = []
#for txt in text_np:
#    l_list.append( len(txt) )
    
#l_list[0:10]

#max(l_list)

In [7]:
maxlen = 207
#training_samples = 1491
#validation_samples = 500
max_words = 100000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(text_np)
sequences = tokenizer.texts_to_sequences(text_np)


In [8]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
data = pad_sequences(sequences, maxlen=maxlen)

print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', train_labels_np.shape)


Found 1635 unique tokens.
Shape of data tensor: (1597, 207)
Shape of label tensor: (1597,)


In [9]:
#indices = np.arange(data.shape[0])
#np.random.shuffle(indices)
#data = data[indices]
#train_labels_np = train_labels_np[indices]
#x_train = data[:training_samples]
#y_train = train_labels_np[:training_samples]
#x_val = data[training_samples: training_samples + validation_samples]
#y_val = train_labels_np[training_samples: training_samples + validation_samples]

In [10]:
label_encoder = LabelEncoder()

y_train_integer_encoded = label_encoder.fit_transform(train_labels_np)
#train_labels_npy_val_integer_encoded = label_encoder.fit_transform(y_val)

#y_train_integer_encoded
#y_train_integer_encodedtrain_labels_np
y_train_one_hot = to_categorical(y_train_integer_encoded, num_classes=7)
#y_val_one_hot = to_categorical(y_val_integer_encoded, num_classes=7)

In [11]:
y_train_integer_encoded

array([6, 2, 6, ..., 5, 5, 4])

In [12]:
y_train_one_hot

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [13]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
model = Sequential()
model.add(Embedding(max_words, 8, input_length=maxlen))
model.add(Flatten())
model.add(Dense(7, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 207, 8)            800000    
_________________________________________________________________
flatten (Flatten)            (None, 1656)              0         
_________________________________________________________________
dense (Dense)                (None, 7)                 11599     
Total params: 811,599
Trainable params: 811,599
Non-trainable params: 0
_________________________________________________________________


In [14]:
history = model.fit(data, y_train_one_hot,
epochs=80,
batch_size=32,
validation_split=.1)

Epoch 1/80
45/45 [==============================] - 1s 16ms/step - loss: 0.3723 - accuracy: 0.5261 - val_loss: 0.5022 - val_accuracy: 0.0000e+00
Epoch 2/80
45/45 [==============================] - 0s 10ms/step - loss: 0.3102 - accuracy: 0.5358 - val_loss: 0.4636 - val_accuracy: 0.0000e+00
Epoch 3/80
45/45 [==============================] - 0s 11ms/step - loss: 0.2992 - accuracy: 0.5358 - val_loss: 0.4976 - val_accuracy: 0.0000e+00
Epoch 4/80
45/45 [==============================] - 0s 11ms/step - loss: 0.2828 - accuracy: 0.5595 - val_loss: 0.5104 - val_accuracy: 0.0000e+00
Epoch 5/80
45/45 [==============================] - 1s 13ms/step - loss: 0.2639 - accuracy: 0.6061 - val_loss: 0.4755 - val_accuracy: 0.0000e+00
Epoch 6/80
45/45 [==============================] - 1s 13ms/step - loss: 0.2441 - accuracy: 0.6360 - val_loss: 0.4192 - val_accuracy: 0.0000e+00
Epoch 7/80
45/45 [==============================] - 1s 13ms/step - loss: 0.2258 - accuracy: 0.6597 - val_loss: 0.4340 - val_accura

In [15]:
modelv2 = Sequential()
modelv2.add(Embedding(max_words, 8, input_length=maxlen))
modelv2.add(Flatten())
modelv2.add(Dense(64, activation='relu'))
modelv2.add(Dense(7, activation='sigmoid'))
modelv2.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
modelv2.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 207, 8)            800000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 1656)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                106048    
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 455       
Total params: 906,503
Trainable params: 906,503
Non-trainable params: 0
_________________________________________________________________


In [16]:
history2= modelv2.fit(data, y_train_one_hot,
epochs=80,
batch_size=32,
validation_split=.1)

Epoch 1/80
45/45 [==============================] - 1s 14ms/step - loss: 0.3661 - accuracy: 0.4760 - val_loss: 0.4923 - val_accuracy: 0.0000e+00
Epoch 2/80
45/45 [==============================] - 0s 10ms/step - loss: 0.3072 - accuracy: 0.5358 - val_loss: 0.5059 - val_accuracy: 0.0000e+00
Epoch 3/80
45/45 [==============================] - 1s 14ms/step - loss: 0.2842 - accuracy: 0.5553 - val_loss: 0.5094 - val_accuracy: 0.0000e+00
Epoch 4/80
45/45 [==============================] - 0s 10ms/step - loss: 0.2474 - accuracy: 0.6312 - val_loss: 0.4007 - val_accuracy: 0.0812
Epoch 5/80
45/45 [==============================] - 0s 10ms/step - loss: 0.2119 - accuracy: 0.6917 - val_loss: 0.4150 - val_accuracy: 0.1187
Epoch 6/80
45/45 [==============================] - 0s 10ms/step - loss: 0.1848 - accuracy: 0.7342 - val_loss: 0.3353 - val_accuracy: 0.5000
Epoch 7/80
45/45 [==============================] - 0s 10ms/step - loss: 0.1655 - accuracy: 0.7724 - val_loss: 0.3355 - val_accuracy: 0.4812
E

In [17]:
train_df

,cpsc_case_number,psu,stratum,weight,treatment_date,age,sex,body_part,diagnosis,primary_mechanism,...,hit_ball_words,five_words,mov_words,heat_words,faint_words,cardio_words,hit_other,hit_player,other_sport,narrative
0,140825963,73,S,82.3076,2014-08-09,85,Female,ALL PARTS BODY,OTHER,cardio/sync,...,1,0,0,0,1,0,0,0,0,85yof passed out as she was playing pickleball...
1,140918710,93,V,15.6716,2014-07-31,57,Male,HEAD,LACERATION,Hit with various obj,...,1,0,0,0,0,0,1,0,0,57 yom hit head on concrete wall while playing...
2,141253842,63,M,99.7040,2014-09-25,73,Male,UPPER TRUNK,OTHER,cardio/sync,...,1,0,0,0,0,2,0,0,0,73yom playing pickelball and developed pain ac...
3,150206597,73,S,78.5926,2015-01-20,78,Female,UPPER TRUNK,OTHER,cardio/sync,...,1,0,0,0,0,2,0,0,0,78yof developed pain across chest while playin...
4,150343352,73,S,80.8381,2015-03-12,64,Female,UPPER TRUNK,OTHER,cardio/sync,...,1,0,0,0,0,2,0,0,0,64yof developed pain in chest while playing pi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1592,190919975,61,V,18.4000,2019-09-08,60,Male,LOWER LEG,"STRAIN, SPRAIN",Other mechanism,...,0,0,2,0,0,0,0,0,0,60yom with sprained lower leg during tennis d...
1593,190929826,63,M,81.1520,2019-09-09,71,Male,KNEE,DISLOCATION,Other mechanism,...,0,0,1,0,0,0,1,0,0,71yom felt a pop in lft knee while playing ten...
1594,190964041,21,V,18.4000,2019-09-26,73,Male,FINGER,"CONTUSIONS, ABR.",Undetermined/unknown,...,0,0,0,0,0,0,0,0,0,"73yom, c/o rt 4th finger swelling noticed afte..."
1595,191115991,2,V,19.5500,2019-10-23,75,Male,SHOULDER,OTHER,Undetermined/unknown,...,0,0,0,0,0,0,0,0,0,75yom was playing tennis yesterday now his sho...


In [18]:
model.save_weights('../Model Objects/keras_outv2.h5')
modelv2.save_weights('../Model Objects/keras_out_extra_layerv2.h5')

In [ ]:
def getModel():
    my_dim = 8
    
    model = Sequential()
    model.add(Embedding(max_words, my_dim, input_length=maxlen))
    model.add(Flatten())
#    model.add(Dense(32, activation='relu'))
    model.add(Dense(7, activation='softmax'))
    
    return model

In [ ]:
# Different configurations to try
epochs_list = [30, 60, 90]
batch_sizes = [16, 32, 64]
#embedding_dim = 8
# Dictionary to store the history of each configuration
history_dict = {}

for epochs in epochs_list:
    for batch_size in batch_sizes:
        # Get a fresh instance of the model for each run
        model = getModel()

        # Compile the model
        model.compile(optimizer='rmsprop',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

        # Train the model
        history = model.fit(x_train, y_train_one_hot,
                            epochs=epochs,
                            batch_size=batch_size,
                            validation_data=(x_val, y_val_one_hot),
                            verbose=2)  # Set verbose to 2 for less output

        # Save the history of this configuration
        history_dict[f"epochs_{epochs}_batch_{batch_size}"] = history.history

        # Optionally, you can print out the final validation accuracy for each configuration
        final_val_accuracy = history.history['val_accuracy'][-1]
        print(f"Epochs: {epochs}, Batch size: {batch_size}, Validation accuracy: {final_val_accuracy}")

# Now you can analyze the history_dict to see which configuration performed best


In [ ]:
history_dict